In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install shap
!pip install feature_engine

In [ ]:
!pip install lime

In [ ]:
!pip install -U imbalanced-learn scikit-learn

In [ ]:
import os
import joblib
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats.mstats import winsorize
from scipy.stats import skew
from sklearn.utils import shuffle
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from feature_engine.selection import RecursiveFeatureAddition
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import BorderlineSMOTE
import shap
from sklearn.metrics import confusion_matrix
import seaborn as sns
import random

In [ ]:
random.seed(42)

I am utilizing this code as guidance, https://github.com/inab-certh/Predicting-COVID-19-severity-through-interpretable-AI-analysis-of-plasma-proteomics/blob/main/Task%202.ipynb

Categorizing ages as follows:
- ages 0 to 40
- ages 41 to 60
- ages 61 to 80
- ages 81 and above

In [ ]:
def bin_age(data):
  age_bins = [0, 40, 60, 80, 100]
  age_labels = [0, 1, 2, 3]

  data['age'] = pd.cut(data['age'], bins=age_bins, labels=age_labels, right=True).astype(int)

  print("ISB Clinical:")
  print(data['age'].value_counts())

In [ ]:
def target_recode_scale(value):
    if value in [1, 2, 3, '1 or 2']:
        return 'mild'
    elif value in [4, 5]:
        return 'moderate'
    elif value in [6, 7]:
        return 'severe'

In [ ]:
def replace_unknown_with_mode(train_column, test_column):
    mode_value = train_column.mode()[0]
    return train_column.replace('Unknown', mode_value), test_column.replace('Unknown', mode_value)

In [ ]:
def get_data_type(data_path, file_type='excel'):
    if 'clinical' in data_path.lower():
        data = pd.read_excel(data_path, keep_default_na=False, na_values=['NA'])
    else:
        if file_type == 'csv':
            data = pd.read_csv(data_path, index_col=0)
        elif file_type == 'excel':
            data = pd.read_excel(data_path)
        elif file_type == 'txt':
            data = pd.read_csv(data_path, sep='\t', index_col=0)
        else:
            raise ValueError("Unsupported file type")
    print("Data loaded successfully.")

    if 'clinical' in data_path.lower():
        data_type = 'Clinical'
    elif 'metabolomics' in data_path.lower():
        data_type = 'Metabolomics'
    elif 'proteomics' in data_path.lower():
        data_type = 'Proteomics'
    else:
        raise ValueError("Unknown data type")

    print(data.columns)
    return data, data_type

In [ ]:
def preprocess_clinical(data, output_dir):
  data.drop('ethnicity', axis=1, inplace=True)
  data.set_index('subject_id', inplace=True)
  relevant_clinical_columns = ['who_severity', 'sex', 'age', 'cigarette_smoking', 'kidney_disease',
                            'chronic_hypertension', 'cancer', 'asthma', 'copd', 'coronary_artery_disease',
                            'respiratory_support']

  data = data[relevant_clinical_columns]
  print("ISB Clinical columns: ", data)

  data['respiratory_support'].replace({'None': 'None'}, inplace=True)
  print("These are the unique respiratory support values:", data['respiratory_support'].unique())
  clin_rows_with_na = data.isna().any(axis=1)

  print("Number of rows with at least one missing value:")
  print("ISB Clinical: ", clin_rows_with_na.sum())

  bin_age(data)
  data['who_severity'] = data['who_severity'].apply(target_recode_scale)
  print("ISB Clinical Severity Value Counts: ", data['who_severity'].value_counts())

  y = data['who_severity'].map({'mild': 0, 'moderate': 1, 'severe': 2})
  print("Target:", y.value_counts())

  X = data.drop(['who_severity'], axis = 1)

  if not os.path.exists(output_dir):
        os.makedirs(output_dir)
  joblib.dump(X, os.path.join(output_dir, 'X_clinical.pkl'))

  return X, y

In [ ]:
def preprocess_metabolomics(data, output_dir):
  data.set_index('subject_id', inplace=True)
  met_rows_with_na = data.isna().any(axis=1)

  print("Number of rows with at least one missing value:")
  print("ISB Metabolomics: ", met_rows_with_na.sum())
  X = data.drop(['Blood Draw', 'Healthy or INCOV', 'age', 'sex', 'BMI'], axis = 1)

  if not os.path.exists(output_dir):
        os.makedirs(output_dir)
  joblib.dump(X, os.path.join(output_dir, 'X_metabolomics.pkl'))

  return X

In [ ]:
def preprocess_proteomics(data, output_dir):
  data.set_index('subject_id', inplace=True)
  pro_rows_with_na = data.isna().any(axis=1)

  print("Number of rows with at least one missing value:")
  print("ISB Proteomics: ", pro_rows_with_na.sum())
  X = data.drop(['Blood Draw', 'Healthy or INCOV', 'age', 'sex', 'BMI'], axis = 1)

  if not os.path.exists(output_dir):
        os.makedirs(output_dir)
  joblib.dump(X, os.path.join(output_dir, 'X_proteomics.pkl'))

  return X

In [ ]:
def data_split(X, y, data_type):
  X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

  X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
  print(f"{data_type} Train Target:", y_train.value_counts(normalize=True))
  return X_train, X_test, X_val, y_test, y_test, y_val

In [ ]:
def save_data_splits(X_train, X_test, X_val, y_train, y_test, y_val, output_dir, X_train_resampled=None, y_train_resampled=None):
    """
    Saves the train-test data splits to the specified directory using joblib.

    Args:
    - X_train (pd.DataFrame or np.ndarray): Training data features.
    - X_test (pd.DataFrame or np.ndarray): Testing data features.
    - y_train (pd.Series or np.ndarray): Training data labels.
    - y_test (pd.Series or np.ndarray): Testing data labels.
    - output_dir (str): Directory path where the data splits will be saved.

    Note:
    - If the output directory does not exist, it will be created.
    """

    assert X_train.shape[0] == y_train.shape[0], "Mismatch in training data and labels."
    assert X_test.shape[0] == y_test.shape[0], "Mismatch in testing data and labels."
    assert X_val.shape[0] == y_val.shape[0], "Mismatch in validation data and labels."

    print("Training set size:", X_train.shape, y_train.shape)
    print("Testing set size:", X_test.shape, y_test.shape)
    print("Validation set size:", X_val.shape, y_val.shape)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    joblib.dump(X_train, os.path.join(output_dir, 'X_train.pkl'))
    joblib.dump(X_test, os.path.join(output_dir, 'X_test.pkl'))
    joblib.dump(X_val, os.path.join(output_dir, 'X_val.pkl'))
    joblib.dump(y_train, os.path.join(output_dir, 'y_train.pkl'))
    joblib.dump(y_test, os.path.join(output_dir, 'y_test.pkl'))
    joblib.dump(y_val, os.path.join(output_dir, 'y_val.pkl'))

    if X_train_resampled is not None and y_train_resampled is not None:
        joblib.dump(X_train_resampled, os.path.join(output_dir, 'X_train_resampled.pkl'))
        joblib.dump(y_train_resampled, os.path.join(output_dir, 'y_train_resampled.pkl'))

In [ ]:
def fill_missing(X_train, X_test, X_val):
  columns_to_fill = ['cancer', 'cigarette_smoking', 'asthma', 'copd', 'coronary_artery_disease']
  for column in columns_to_fill:
    if column in X_train.columns:
      X_train[column], X_test[column] = replace_unknown_with_mode(X_train[column], X_test[column])
      X_train[column], X_val[column] = replace_unknown_with_mode(X_train[column], X_val[column])
  return X_train, X_test, X_val

In [ ]:
def encode_ordinal_columns(X_train, X_test, X_val):
  cigarette_ordering = [['Never', 'Former', 'Current']]
  respiratory_support_ordering = [['None', 'Other', 'Nasal cannula', 'High flow nasal cannula (HFNC)']]
  encoder_cigarette = OrdinalEncoder(categories=cigarette_ordering)
  encoder_respiratory_support = OrdinalEncoder(categories=respiratory_support_ordering)
  columns_to_encode = ['cigarette_smoking', 'respiratory_support']

  for column in columns_to_encode:
      if column in X_train.columns:
        if column == 'cigarette_smoking':
            X_train['cigarette_smoking'] = encoder_cigarette.fit_transform(X_train[[column]])
            X_test['cigarette_smoking'] = encoder_cigarette.transform(X_test[[column]])
            X_val['cigarette_smoking'] = encoder_cigarette.transform(X_val[[column]])
        elif column == 'respiratory_support':
            X_train['respiratory_support'] = encoder_respiratory_support.fit_transform(X_train[[column]])
            X_test['respiratory_support'] = encoder_respiratory_support.transform(X_test[[column]])
            X_val['respiratory_support'] = encoder_respiratory_support.transform(X_val[[column]])
  return X_train, X_test, X_val

In [ ]:
def encode_binary_columns(X_train, X_test, X_val):
  binary_columns = ['sex', 'kidney_disease', 'chronic_hypertension', 'cancer', 'asthma', 'copd', 'coronary_artery_disease']
  binary_mapping = {'Male': 1, 'Yes': 1, 'Female': 0, 'No': 0}

  for column in binary_columns:
    if column in X_train.columns:
      X_train[column] = X_train[column].map(binary_mapping)
      X_val[column] = X_val[column].map(binary_mapping)
      X_test[column] = X_test[column].map(binary_mapping)
  return X_train, X_test, X_val

In [ ]:
def corr_matrix(X_train, output_dir):
    corr_matrix = X_train.corr().abs()

    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, cmap='coolwarm', center=0, linewidths=1, annot=False, fmt=".2f")

    output_file = os.path.join(output_dir, 'correlation_matrix.png')

    plt.savefig(output_file)
    plt.show()

In [ ]:
def multicollinearity(X_test, output_dir):
    df_const = add_constant(X_test)

    vif_data = pd.DataFrame()
    vif_data["Variable"] = df_const.columns
    vif_data["VIF"] = [variance_inflation_factor(df_const.values, i) for i in range(df_const.shape[1])]

    print(vif_data)

    output_path = os.path.join(output_dir, "vif_data.csv")

    vif_data.to_csv(output_path, index=False)

In [ ]:
proteomics_scaling_selection = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('selector', RFE(estimator=LogisticRegression(), n_features_to_select=100))
])

metabolomics_scaling_selection = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('selector', RFE(estimator=LogisticRegression(), n_features_to_select=100))
])

In [ ]:
def omics_pipeline(X_train, X_val, X_test, y_train, data_type):
    if data_type == 'Clinical':
        print("Clinical data received. Skipping preprocessing.")
        return X_train, X_val, X_test

    elif data_type == 'Metabolomics':
        scaling_selection = metabolomics_scaling_selection
    elif data_type == 'Proteomics':
        scaling_selection = proteomics_scaling_selection
    else:
        raise ValueError(f"Data type {data_type} is not supported for this preprocessing.")

    X_train_preprocessed = scaling_selection.fit_transform(X_train, y_train)
    selected_features = X_train.columns[scaling_selection.named_steps['selector'].support_]

    X_val_preprocessed = scaling_selection.transform(X_val)
    X_test_preprocessed = scaling_selection.transform(X_test)

    X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=selected_features, index=X_train.index)
    X_val_preprocessed = pd.DataFrame(X_val_preprocessed, columns=selected_features, index=X_val.index)
    X_test_preprocessed = pd.DataFrame(X_test_preprocessed, columns=selected_features, index=X_test.index)

    return X_train_preprocessed, X_val_preprocessed, X_test_preprocessed

In [ ]:
def pca_process(X_train_preprocessed, X_val_preprocessed, X_test_preprocessed, data_type):
  original_columns = X_train_preprocessed.columns.tolist()

  pca = PCA(.90)

  X_train_preprocessed = pd.DataFrame(pca.fit_transform(X_train_preprocessed), index=X_train_preprocessed.index)
  X_test_preprocessed = pd.DataFrame(pca.transform(X_test_preprocessed), index=X_test_preprocessed.index)
  X_val_preprocessed = pd.DataFrame(pca.transform(X_val_preprocessed), index=X_val_preprocessed.index)

  loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

  loading_matrix = pd.DataFrame(loadings, index=original_columns)
  loading_matrix

  load_trans = loading_matrix.T
  loading_pow = {}


  for i in load_trans.columns:
      magnitude = load_trans[i].pow(2).sum()
      loading_pow[i] = magnitude

  file_path_matrix = 'loading_matrix_' + data_type + '.xlsx'
  loading_matrix.to_excel(file_path_matrix)

  file_path_pow = 'loadingspow' + data_type + '.xlsx'
  pd.DataFrame.from_dict(loading_pow, orient='index').to_excel(file_path_pow)

  return X_train_preprocessed, X_val_preprocessed, X_test_preprocessed

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from numpy import mean
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

models = {
    'logistic_regression': LogisticRegression(),
    'decision_tree': DecisionTreeClassifier(),
    'rand_forest': RandomForestClassifier(),
    'extra_trees': ExtraTreesClassifier(),
    'mlp': MLPClassifier(),
    'svm': SVC(),
    'ada_boost': AdaBoostClassifier(),
    'xgb': xgb.XGBClassifier(),
    'gbm': GradientBoostingClassifier(),
    'qda': QuadraticDiscriminantAnalysis(),
}

log_params = {
    'logistic_regression__penalty': ['l1', 'l2'],
    'logistic_regression__C': [0.1, 1, 10],
    'logistic_regression__solver': ['liblinear', 'saga'],
    'logistic_regression__max_iter': [100, 200, 300, 400, 500, 600, 700, 800, 900],
    'logistic_regression__class_weight': [None, 'balanced']
}


dt_params = {
    'decision_tree__criterion': ['gini', 'entropy'],
    'decision_tree__max_depth': [None,1, 5, 10, 50, 100],
    'decision_tree__min_samples_split': [2, 5, 10],
    'decision_tree__min_samples_leaf': [1, 2, 4],
    'decision_tree__splitter':['best','random'],
    'decision_tree__class_weight': [None, 'balanced']
}

rf_params = {
    'rand_forest__n_estimators':[50,100,500,1000,2000],
    'rand_forest__criterion': ['gini', 'entropy'],
    'rand_forest__max_depth': [None, 5, 10, 20],
    'rand_forest__min_samples_split': [2, 5, 10],
    'rand_forest__min_samples_leaf': [1, 2, 4],
    'rand_forest__class_weight': [None, 'balanced']
}

et_params = {
    'extra_trees__n_estimators': [100, 200, 500, 1000, 2000],
    'extra_trees__criterion': ['gini', 'entropy'],
    'extra_trees__max_depth': [None, 5, 10],
    'extra_trees__min_samples_split': [2, 5, 10],
    'extra_trees__min_samples_leaf': [1, 2, 4],
    'extra_trees__class_weight': [None, 'balanced']
}

mlp_params = {
    'mlp__hidden_layer_sizes': [(100,), (100, 50), (50, 50), (50,50,50)],
    'mlp__activation': ['relu', 'tanh', 'logistic', 'identity'],
    'mlp__solver': ['adam', 'sgd', 'lbfgs'],
    'mlp__learning_rate':['constant','adaptive'],
    'mlp__alpha': [0.0001, 0.001, 0.01, 0.05],
    'mlp__max_iter': [100, 200, 300, 400, 500, 600, 700, 800, 900]
}

svm_params = {
    'svm__C':[0.1, 1, 2, 5, 10, 50, 100, 500],
    'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'svm__gamma': ['scale', 'auto'],
    'svm__probability': [True],
    'svm__class_weight': [None, 'balanced'],
    'svm__max_iter': [100, 200, 300, 400, 500, 600, 700, 800, 900]
}

ada_params = {
    'ada_boost__n_estimators': [10, 20, 30, 50, 100, 200, 500, 1000],
    'ada_boost__learning_rate': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
}

xgb_params = {
    'xgb__n_estimators': [100, 200, 500],
    'xgb__learning_rate': [0.01, 0.03, 0.06, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7],
    'xgb__max_depth': [3, 5, 7, 9, 11],
    'xgb__subsample': [0.8, 1.0],
    'xgb__colsample_bytree': [0.8, 1.0],
    'xgb__gamma': [0, 0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6, 51.2, 102.4, 200],
    'xgb__reg_alpha': [0, 0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6, 51.2, 102.4, 200],
    'xgb__reg_lambda': [0, 0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6, 51.2, 102.4, 200]
}

gbm_params = {
    'gbm__n_estimators': [50, 100, 200, 500, 1000, 2000],
    'gbm__learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
    'gbm__max_depth': [3, 5, 7],
    'gbm__subsample': [0.8, 1.0],
    'gbm__max_features': ['sqrt', 'log2']
}

qda_params = {
    'qda__reg_param': [0, 0.5, 1]
}



params = {
    'logistic_regression': log_params,
    'decision_tree': dt_params,
    'rand_forest': rf_params,
    'extra_trees': et_params,
    'mlp': mlp_params,
    'svm': svm_params,
    'ada_boost': ada_params,
    'xgb': xgb_params,
    'gbm': gbm_params,
    'qda': qda_params,
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import BorderlineSMOTE, SVMSMOTE, RandomOverSampler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
import time
import matplotlib.pyplot as plt


def model_search(models, params, over, under, X_train, y_train, X_test, y_test, X_val, y_val):
    print(f"Number of models: {len(models)}")
    training_times = []
    max_score = 0
    max_model = None
    max_model_params = None
    estimators_modelsearch = pd.DataFrame()
    models_est = []
    parameters = []
    lscore = []
    lroc = []
    lfscore_macro = []
    lfscore_weighted = []
    lprecision = []
    lrecall = []
    lrecall_severe = []
    lclass_report = []
    lcvscore = []
    lscore_val = []
    lroc_val = []
    lfscore_macro_val = []
    lfscore_weighted_val = []
    lprecision_val = []
    lrecall_val = []
    lrecall_severe_val = []
    lclass_report_val = []

    scoring = {
        'balanced_accuracy': 'balanced_accuracy',
        'accuracy': 'accuracy',
        'roc_auc': 'roc_auc_ovr',
        'precision': 'precision_macro',
        'recall_macro': make_scorer(recall_score, average='macro'),
        'f1_macro': make_scorer(f1_score, average='macro'),
        'f1_weighted': make_scorer(f1_score, average='weighted')
    }

    cv = StratifiedKFold(10)


    for i, j in models.items():
        start_time = time.time()

        try:
            pipeline = Pipeline([('over', over), ('under', under), (i, j)])

            rs = RandomizedSearchCV(
                estimator=pipeline,
                param_distributions=params[i],
                scoring=scoring,
                refit='roc_auc',
                cv=cv,
                n_iter=10,
                random_state=42,
                n_jobs=-1
            )

            rs.fit(X_train, y_train)

        except Exception as e:
            print(f"Error while fitting model {i}: {e}")
            continue



        y_pred = rs.predict(X_test)
        class_report = classification_report(y_test, y_pred)
        print(f"Classification Report for {i}:\n", class_report)

        y_pred_val = rs.predict(X_val)
        class_report_val = classification_report(y_val, y_pred_val)
        print(f"Validation Classification Report for {i}:\n", class_report_val)

        try:
            y_pred_proba = rs.predict_proba(X_test)
            lroc.append(roc_auc_score(y_test, y_pred_proba, multi_class='ovr'))
        except AttributeError:
            print(f"{i} does not support probability prediction. ROC AUC score cannot be computed.")
            lroc.append(None)

        try:
            y_pred_proba_val = rs.predict_proba(X_val)
            lroc_val.append(roc_auc_score(y_val, y_pred_proba_val, multi_class='ovr'))
        except AttributeError:
            print(f"{i} does not support probability prediction. ROC AUC score cannot be computed for validation set.")
            lroc_val.append(None)



        lscore.append(accuracy_score(y_test, y_pred))
        lscore_val.append(accuracy_score(y_val, y_pred_val))
        lcvscore.append(rs.best_score_)

        from sklearn.exceptions import UndefinedMetricWarning
        import warnings

        try:
            lprecision.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
            lprecision_val.append(precision_score(y_val, y_pred_val, average='macro', zero_division=0))
        except UndefinedMetricWarning:
            print(f"Warning encountered for model {i}")
            print("Test predictions:", y_pred)
            print("Validation predictions:", y_pred_val)


        lrecall.append(recall_score(y_test, y_pred, average='macro'))
        lrecall_val.append(recall_score(y_val, y_pred_val, average='macro'))

        lfscore_macro.append(f1_score(y_test, y_pred, average='macro'))
        lfscore_macro_val.append(f1_score(y_val, y_pred_val, average='macro'))

        lfscore_weighted.append(f1_score(y_test, y_pred, average='weighted'))
        lfscore_weighted_val.append(f1_score(y_val, y_pred_val, average='weighted'))

        lrecall_severe.append(recall_score(y_test, y_pred, labels=[2], average='macro'))
        lrecall_severe_val.append(recall_score(y_val, y_pred_val, labels=[2], average='macro'))

        lclass_report.append(class_report)
        lclass_report_val.append(class_report_val)

        if lscore[-1] > max_score:
            max_score = lscore[-1]
            max_model = rs.best_estimator_
            max_model_params = rs.best_params_

        models_est.append(i)
        parameters.append(rs.best_params_)

    estimators_modelsearch['Models'] = models_est
    estimators_modelsearch['Best parameters'] = parameters
    estimators_modelsearch['ROC AUC'] = lroc
    estimators_modelsearch['ROC AUC Validation'] = lroc_val
    estimators_modelsearch['F1-score (macro)'] = lfscore_macro
    estimators_modelsearch['F1-score (macro) Validation'] = lfscore_macro_val
    estimators_modelsearch['F1-score (weighted)'] = lfscore_weighted
    estimators_modelsearch['F1-score (weighted) Validation'] = lfscore_weighted_val
    estimators_modelsearch['Precision'] = lprecision
    estimators_modelsearch['Precision Validation'] = lprecision_val
    estimators_modelsearch['Recall'] = lrecall
    estimators_modelsearch['Recall Severe'] = lrecall_severe
    estimators_modelsearch['Recall Severe Validation'] = lrecall_severe_val
    estimators_modelsearch['Score'] = lscore
    estimators_modelsearch['Score Validation'] = lscore_val
    estimators_modelsearch['CV Score'] = lcvscore
    print(len(estimators_modelsearch))

    plt.figure(figsize=(10,5))
    plt.bar(models_est, lscore, label='Test Score')
    plt.bar(models_est, lscore_val, label='Validation Score', alpha=0.5)
    plt.legend()
    plt.title("Performance Comparison")
    plt.show()

    return [max_score, max_model, max_model_params], estimators_modelsearch

In [ ]:
def main(data_paths):
  y = None
  for data_path in data_paths:
    data, data_type = get_data_type(data_path, file_type='excel')
    if data_type == 'Clinical':
        output_dir = os.path.join('covid_baseline_classifier_outputs', data_type)
        X, y = preprocess_clinical(data, output_dir)
    elif data_type == 'Metabolomics':
        if y is None:
          raise ValueError("Clinical data must be processed first to set 'y'.")
        output_dir = os.path.join('covid_baseline_classifier_outputs', data_type)
        X = preprocess_metabolomics(data, output_dir)
    elif data_type == 'Proteomics':
        if y is None:
          raise ValueError("Clinical data must be processed first to set 'y'.")
        output_dir = os.path.join('covid_baseline_classifier_outputs', data_type)
        X = preprocess_proteomics(data, output_dir)



    print(np.unique(y))
    print("X shape:", X.shape)
    print("y shape:", y.shape)
    print("Size before first split:", X.shape, y.shape)
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print("Size after first split - Test set:", X_test.shape, y_test.shape)

    print("Size before second split:", X_temp.shape, y_temp.shape)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
    print("Sizes after second split - Train and Val set:", X_train.shape, y_train.shape, X_val.shape, y_val.shape)

    print(f"{data_type} Train Target:", y_train.value_counts(normalize=True))

    X_train = X_train.copy()
    X_test = X_test.copy()
    X_val = X_val.copy()

    X_train, X_test, X_val = fill_missing(X_train, X_test, X_val)
    X_train, X_test, X_val = encode_ordinal_columns(X_train, X_test, X_val)


    if 'age' in X_train.columns:
      age_train = X_train.pop('age')
      X_train['age'] = age_train
      X_train.head(5)
      age_val = X_val.pop('age')
      X_val['age'] = age_val
      age_test = X_test.pop('age')
      X_test['age'] = age_test

    #for data_path in data_paths:
    if data_type == 'Clinical':
        X_train, X_test, X_val = encode_binary_columns(X_train, X_test, X_val)
        corr_matrix(X_train, output_dir)
        multicollinearity(X_test, output_dir)

        print("Class distribution before resampling:", np.bincount(y_train))
        over = BorderlineSMOTE(sampling_strategy='auto')
        under = RandomUnderSampler(sampling_strategy='majority')

        pipeline = Pipeline([
            ('over', over),
            ('under', under)
        ])

        X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train, y_train)

        print("Class distribution after resampling:", np.bincount(y_train_resampled))

        results, df = model_search(
          models=models,
          params=params,
          over=over,
          under=under,
          X_train=X_train,
          y_train=y_train,
          X_test=X_test,
          y_test=y_test,
          X_val=X_val,
          y_val=y_val
        )
        save_data_splits(X_train, X_test, X_val, y_train, y_test, y_val, os.path.join('covid_baseline_classifier_outputs', data_type), X_train_resampled, y_train_resampled)
        file_path = data_type + '_model_training_results.xlsx'
        df.to_excel(file_path)

    elif data_type == 'Metabolomics':
        X_train_preprocessed, X_val_preprocessed, X_test_preprocessed = omics_pipeline(X_train, X_val, X_test, y_train, data_type)
        X_train_preprocessed, X_val_preprocessed, X_test_preprocessed = pca_process(X_train_preprocessed, X_val_preprocessed, X_test_preprocessed, data_type)
        print("Class distribution before resampling:", np.bincount(y_train))
        over = BorderlineSMOTE(sampling_strategy='auto')
        under = RandomUnderSampler(sampling_strategy='majority')

        pipeline = Pipeline([
            ('over', over),
            ('under', under)
        ])

        X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_preprocessed, y_train)

        print("Class distribution after resampling:", np.bincount(y_train_resampled))

        results, df = model_search(
          models=models,
          params=params,
          over=over,
          under=under,
          X_train=X_train_preprocessed,
          y_train=y_train,
          X_test=X_test_preprocessed,
          y_test=y_test,
          X_val=X_val_preprocessed,
          y_val=y_val
        )

        file_path = data_type + '_model_training_results.xlsx'
        save_data_splits(X_train_resampled, X_test_preprocessed, X_val_preprocessed, y_train_resampled, y_test, y_val, os.path.join('covid_baseline_classifier_outputs', data_type))
        df.to_excel(file_path)

    elif data_type == 'Proteomics':
        X_train_preprocessed, X_val_preprocessed, X_test_preprocessed = omics_pipeline(X_train, X_val, X_test, y_train, data_type)
        X_train_preprocessed, X_val_preprocessed, X_test_preprocessed = pca_process(X_train_preprocessed, X_val_preprocessed, X_test_preprocessed, data_type)
        print("Class distribution before resampling:", np.bincount(y_train))
        over = BorderlineSMOTE(sampling_strategy='auto')
        under = RandomUnderSampler(sampling_strategy='majority')

        pipeline = Pipeline([
            ('over', over),
            ('under', under)
        ])

        X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_preprocessed, y_train)

        print("Class distribution after resampling:", np.bincount(y_train_resampled))

        results, df = model_search(
          models=models,
          params=params,
          over=over,
          under=under,
          X_train=X_train_preprocessed,
          y_train=y_train,
          X_test=X_test_preprocessed,
          y_test=y_test,
          X_val=X_val_preprocessed,
          y_val=y_val
        )

        file_path = data_type + '_model_training_results.xlsx'
        save_data_splits(X_train_resampled, X_test_preprocessed, X_val_preprocessed, y_train_resampled, y_test, y_val, os.path.join('covid_baseline_classifier_outputs', data_type))
        df.to_excel(file_path)


  return X_train_preprocessed, X_val_preprocessed, X_test_preprocessed, y_train, y_test, y_val

In [ ]:
data_paths = ["/content/drive/MyDrive/Master's Thesis/COVID-19/final-project/final_data/ISB/isb_clinical.xlsx",
              "/content/drive/MyDrive/Master's Thesis/COVID-19/final-project/final_data/ISB/isb_metabolomics.xlsx",
              "/content/drive/MyDrive/Master's Thesis/COVID-19/final-project/final_data/ISB/isb_proteomics.xlsx"
              ]
main(data_paths)

In [ ]:
!zip -r covid_baseline_classifier_outputs.zip covid_baseline_classifier_outputs

## Interpretability on Individual Models

{'xgb__subsample': 1.0, 'xgb__reg_lambda': 0.2, 'xgb__reg_alpha': 0.8, 'xgb__n_estimators': 200, 'xgb__max_depth': 3, 'xgb__learning_rate': 0.25, 'xgb__gamma': 0.2, 'xgb__colsample_bytree': 0.8}

In [ ]:
proteomics_model = xgb.XGBClassifier(
        subsample = 1.0,
        reg_lambda = 0.2,
        reg_alpha = 0.8,
        n_estimators = 200,
        max_depth = 3,
        learning_rate = 0.25,
        gamma = 0.2,
        colsample_bytree = 0.8
)

metabolomics_model = SVC(
        probability = True,
        max_iter = 700,
        kernel = 'rbf',
        gamma = 'scale',
        class_weight = 'balanced',
        C = 500,
)

clinical_model = RandomForestClassifier(
    n_estimators = 50,
    min_samples_split = 10,
    min_samples_leaf =  2,
    max_depth = 20,
    criterion = 'entropy',
    class_weight = None
)

In [ ]:
def load_pre_split_data():
        X_train_pro = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Proteomics/X_train.pkl')
        y_train_pro = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Proteomics/y_train.pkl')
        X_test_pro = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Proteomics/X_test.pkl')
        y_test_pro = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Proteomics/y_test.pkl')
        X_val_pro = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Proteomics/X_val.pkl')
        y_val_pro = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Proteomics/y_val.pkl')

        X_train_met = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Metabolomics/X_train.pkl')
        y_train_met = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Metabolomics/y_train.pkl')
        X_test_met = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Metabolomics/X_test.pkl')
        y_test_met = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Metabolomics/y_test.pkl')
        X_val_met = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Metabolomics/X_val.pkl')
        y_val_met = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Metabolomics/y_val.pkl')

        X_train_clin = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Clinical/X_train.pkl')
        y_train_clin = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Clinical/y_train.pkl')
        X_test_clin = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Clinical/X_test.pkl')
        y_test_clin = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Clinical/y_test.pkl')
        X_val_clin = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Clinical/X_val.pkl')
        y_val_clin = joblib.load('/content/drive/MyDrive/covid_baseline_classifier_outputs/Clinical/y_val.pkl')

        return X_train_met, y_train_met, X_test_met, y_test_met, X_val_met, y_val_met, X_train_pro, y_train_pro, X_test_pro, y_test_pro, X_val_pro, y_val_pro, X_train_clin, y_train_clin, X_test_clin, y_test_clin, X_val_clin, y_val_clin

In [ ]:
def load_model(data_type):
  if data_type == "Proteomics":
      model = proteomics_model

      explainer_type = 'tree'
      shap_filename = 'pro_shap.svg'
      lime_filename = 'pro_lime.html'
      cm_filename = 'pro_confusion_matrix.png'
      output_file_name = 'pro_feature_importance.xlsx'
      index = 10
  elif data_type == "Metabolomics":
      model = metabolomics_model

      explainer_type = 'kernel'
      shap_filename = 'met_shap.svg'
      lime_filename = 'met_lime.html'
      cm_filename = 'met_confusion_matrix.png'
      output_file_name = 'met_feature_importance.xlsx'
      index = 10
  elif data_type == "Clinical":
      model = clinical_model
      loading_matrix = None
      explainer_type = 'tree'
      shap_filename = 'clin_shap.svg'
      lime_filename = 'clin_lime.html'
      cm_filename = 'clin_confusion_matrix.png'
      output_file_name = 'clin_feature_importance.xlsx'
      index = 9
  else:
      raise ValueError(f"Unexpected data_type: {data_type}")
  return model, explainer_type, shap_filename, lime_filename, cm_filename, output_file_name, index

In [ ]:
import os
import shap
import numpy as np
import pandas as pd
from lime import lime_tabular
import matplotlib.pyplot as plt

np.random.seed(42)

def train_and_explain_model(model, X_train, y_train, X_test, y_test, loading_matrix, explainer_type, shap_filename, lime_filename, cm_filename, output_dir, output_filename, i):
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    joblib.dump(model, os.path.join(output_dir, 'trained_model.pkl'))

    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')

    plt.savefig(cm_filename, dpi=300)
    plt.close()

    if explainer_type == "tree":
        explainer = shap.TreeExplainer(model)
    elif explainer_type == "kernel":
        explainer = shap.KernelExplainer(model.predict_proba, X_train)
    elif explainer_type == "linear":
        explainer = shap.LinearExplainer(model, X_train)
    else:
        raise ValueError("Invalid explainer type")

    shap_values = explainer.shap_values(X_train)
    shap.summary_plot(shap_values, X_train, feature_names=X_train.columns, max_display=20, show=False)
    plt.savefig(shap_filename)
    plt.close()


    if isinstance(shap_values, list):
        vals = np.abs(shap_values[0]).mean(0)
    else:
        vals = np.abs(shap_values).mean(0)

    feature_importance = pd.DataFrame(list(zip(X_train.columns, vals)), columns=['col_name', 'feature_importance_vals'])
    feature_importance.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)
    feature_importance.to_excel(output_filename)

    # loadings
    if loading_matrix is not None:
        assert isinstance(loading_matrix, pd.DataFrame), "loading_matrix should be a pandas DataFrame."
        top_features = [i for i in list(feature_importance['col_name'])[:30] if str(i).isdigit()]
        print(top_features)
        loading_trans = loading_matrix[top_features].T
        loading_pow = {}
        for column in loading_trans.columns:
            magnitude = loading_trans[column].pow(2).sum()
            loading_pow[column] = magnitude
        pd.DataFrame.from_dict(loading_pow, orient='index').to_excel(output_filename.replace('.xlsx', '_loading_pow.xlsx'))

    explainer = lime_tabular.LimeTabularExplainer(X_train.to_numpy(), feature_names=X_train.columns, class_names=["Mild", "Moderate", "Severe"], mode='classification', random_state=42)
    exp = explainer.explain_instance(X_test.iloc[i].to_numpy(), model.predict_proba, num_features=10)
    exp.save_to_file(os.path.join(os.getcwd(), lime_filename))

    observation = X_test.iloc[i]
    observation_df = pd.DataFrame(observation).transpose()
    observation_df['who_severity'] = y_test.iloc[i]
    print(observation_df)

    y_pred_proba = model.predict_proba(X_test)
    return y_pred_proba


In [ ]:
data_paths = ['baseline_classifier_outputs/Clinical',
              'baseline_classifier_outputs/Metabolomics',
              'baseline_classifier_outputs/Proteomics']

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

X_train_met, y_train_met, X_test_met, y_test_met, X_val_met, y_val_met, X_train_pro, y_train_pro, X_test_pro, y_test_pro, X_val_pro, y_val_pro, X_train_clin, y_train_clin, X_test_clin, y_test_clin, X_val_clin, y_val_clin = load_pre_split_data()
X_train_met.columns = X_train_met.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '')
X_test_met.columns = X_test_met.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '')
X_train_met.columns = X_train_met.columns.astype(str)
X_test_met.columns = X_test_met.columns.astype(str)

np.random.seed(42)

output_dir_met = "output/metabolomics"
output_dir_pro = "output/proteomics"

met_pred_proba_test = train_and_explain_model(
  metabolomics_model, X_train_met, y_train_met, X_test_met, y_test_met,
  'kernel', 'met_shap.svg', 'met_lime.html', 'met_confusion_matrix.png',
  output_dir_met, 'met_feature_importance.xlsx', 10
)

pro_pred_proba_test = train_and_explain_model(
  proteomics_model, X_train_pro, y_train_pro, X_test_pro, y_test_pro,
  'tree', 'pro_shap.svg', 'pro_lime.html', 'pro_confusion_matrix.png',
  output_dir_pro, 'pro_feature_importance.xlsx', 10
)

clin_pred_proba_test = train_and_explain_model(
  metabolomics_model, X_train_met, y_train_met, X_test_met, y_test_met,
  'kernel', 'met_shap.svg', 'met_lime.html', 'met_confusion_matrix.png',
  output_dir_met, 'met_feature_importance.xlsx', 10
)

In [ ]:
shap_values = np.load("/content/output/metabolomics/met_feature_importance.xlsx_shap_values.npy")

feature_names = X_train_met.columns

print("SHAP values shape:", shap_values.shape)

# loop over all classes (3 classes in this case)
for class_idx in range(shap_values.shape[2]):
    # mean absolute SHAP values for the current class
    class_shap_values = np.abs(shap_values[:, :, class_idx]).mean(axis=0)

    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'importance': class_shap_values
    })

    # sort by importance
    feature_importance = feature_importance.sort_values(by='importance', ascending=False)

    excel_filename = f"met_feature_importance_class_{class_idx}.xlsx"
    feature_importance.to_excel(excel_filename, index=False)

    print(f"Feature Importance for Class {class_idx} saved to {excel_filename}")

In [ ]:
shap_values = np.load("/content/output/proteomics/pro_feature_importance.xlsx_shap_values.npy")

feature_names = X_train_pro.columns

print("SHAP values shape:", shap_values.shape)

# loop over all classes (3 classes in this case)
for class_idx in range(shap_values.shape[2]):
    # mean absolute SHAP values for the current class
    class_shap_values = np.abs(shap_values[:, :, class_idx]).mean(axis=0)

    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'importance': class_shap_values
    })

    # sort by importance
    feature_importance = feature_importance.sort_values(by='importance', ascending=False)

    excel_filename = f"pro_feature_importance_class_{class_idx}.xlsx"
    feature_importance.to_excel(excel_filename, index=False)

    print(f"Feature Importance for Class {class_idx} saved to {excel_filename}")

In [ ]:
shap_values = np.load("/content/output/clinical/clin_feature_importance.xlsx_shap_values.npy")

feature_names = X_train_clin.columns

print("SHAP values shape:", shap_values.shape)

# loop over all classes (3 classes in this case)
for class_idx in range(shap_values.shape[2]):
    # mean absolute SHAP values for the current class
    class_shap_values = np.abs(shap_values[:, :, class_idx]).mean(axis=0)

    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'importance': class_shap_values
    })

    # sort by importance
    feature_importance = feature_importance.sort_values(by='importance', ascending=False)

    excel_filename = f"clin_feature_importance_class_{class_idx}.xlsx"
    feature_importance.to_excel(excel_filename, index=False)

    print(f"Feature Importance for Class {class_idx} saved to {excel_filename}")

## Stacking

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class CustomModel(BaseEstimator, ClassifierMixin):
    def __init__(self, model, feature_indices):
        self.model = model
        self.feature_indices = feature_indices

    def fit(self, X, y=None):
        self.model.fit(X[:, self.feature_indices], y)
        self.classes_ = self.model.classes_  # set the classes_ attribute
        return self

    def predict_proba(self, X):
        return self.model.predict_proba(X[:, self.feature_indices])

    def predict(self, X):
        return self.model.predict(X[:, self.feature_indices])

    def get_params(self, deep=True):
        return {"model": self.model, "feature_indices": self.feature_indices}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)


In [ ]:
X_train = np.concatenate([X_train_met, X_train_pro, X_train_clin], axis=1)
X_test = np.concatenate([X_test_met, X_test_pro, X_test_clin], axis=1)
X_val = np.concatenate([X_val_met, X_val_pro, X_val_clin], axis=1)
y_train = y_train_clin
y_test = y_test_clin
y_val = y_val_clin

# feature indices for each modality (Metabolomics and Proteomics)

metabolomics_feature_indices = list(range(X_train_met.shape[1]))
proteomics_feature_indices = list(range(X_train_met.shape[1], X_train_met.shape[1] + X_train_pro.shape[1]))
clinical_feature_indices = list(range(X_train_met.shape[1] + X_train_pro.shape[1], X_train_clin.shape[1]))

print("Metabolomics Feature Indices:", metabolomics_feature_indices)
print("Proteomics Feature Indices:", proteomics_feature_indices)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score

base_models = [
    ('svc', CustomModel(metabolomics_model, metabolomics_feature_indices)),
    ('xgb', CustomModel(proteomics_model, proteomics_feature_indices)),
    ('rf', CustomModel(clinical_model, clinical_feature_indices)),
]

stacked_models = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression(multi_class='multinomial', penalty = 'l2', random_state=42))

print(f"Starting model training at: {time.ctime()}")
start_time = time.time()

stacked_models.fit(X_train, y_train)

y_pred = stacked_models.predict(X_test)
y_pred_proba = stacked_models.predict_proba(X_test)
y_pred_val = stacked_models.predict(X_val)
y_pred_proba_val = stacked_models.predict_proba(X_val)

score = accuracy_score(y_test, y_pred)
score_val = accuracy_score(y_val, y_pred_val)
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
roc_auc_val = roc_auc_score(y_val, y_pred_proba_val, multi_class='ovr')
fscore_macro = f1_score(y_test, y_pred, average='macro')
fscore_macro_val = f1_score(y_val, y_pred_val, average='macro')
fscore_weighted = f1_score(y_test, y_pred, average='weighted')
fscore_weighted_val = f1_score(y_val, y_pred_val, average='weighted')
precision = precision_score(y_test, y_pred, average='macro')
precision_val = precision_score(y_val, y_pred_val, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
recall_val = recall_score(y_val, y_pred_val, average='macro')
recall_severe = recall_score(y_test, y_pred, labels=[2], average='macro')
recall_severe_val = recall_score(y_val, y_pred_val, labels=[2], average='macro')


end_time = time.time()
processing_time = end_time - start_time

X_full = np.concatenate([X_train, X_test, X_val], axis=0)
y_full = np.concatenate([y_train, y_test, y_val], axis=0)

cvscore = cross_val_score(stacked_models, X_full, y_full, cv=StratifiedKFold(10), scoring='roc_auc_ovr').mean()

estimators_model = pd.DataFrame()
estimators_model['Models'] = ['Stacking Ensemble Model']
estimators_model['ROC AUC'] = [roc_auc]
estimators_model['ROC AUC Validation'] = [roc_auc_val]
estimators_model['F1-score (macro)'] = [fscore_macro]
estimators_model['F1-score (macro) Validation'] = [fscore_macro_val]
estimators_model['F1-score (weighted)'] = [fscore_weighted]
estimators_model['F1-score (weighted) Validation'] = [fscore_weighted_val]
estimators_model['Precision'] = [precision]
estimators_model['Precision Validation'] = [precision_val]
estimators_model['Recall'] = [recall]
estimators_model['Recall Validation'] = [recall_val]
estimators_model['Recall Severe'] = [recall_severe]
estimators_model['Recall Severe Validation'] = [recall_severe_val]
estimators_model['Score'] = [score]
estimators_model['Score Validation'] = [score_val]
estimators_model['CV Score'] = [cvscore]
estimators_model['Training Time'] = [processing_time]

In [ ]:
print(estimators_model)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.savefig('confusion_matrix.png', dpi=300)
plt.show()

In [ ]:
for data_path in data_paths:
    data_type = data_path.split('/')[1]
    print("Data type:", data_type)
    output_dir = os.path.join('covid_baseline_classifier_outputs', data_type)
    X_train, y_train, X_test, y_test, X_val, y_val = load_pre_split_data(data_type)
    model, explainer_type, shap_filename, lime_filename, cm_filename, output_file_name, index = load_model(data_type)
    train_and_explain_model(model, X_train, y_train, X_test, y_test, explainer_type, shap_filename, lime_filename, cm_filename, output_dir, output_file_name, index)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


data = pd.read_excel('/content/met_feature_importance_class_2.xlsx')

data = data.sort_values(by='importance', ascending=True).head(15)

plt.figure(figsize=(8, 6))
plt.scatter(data['importance'], data['feature'], color='purple', s=100)

plt.title('Importance of Plasma Metabolites for Severe Class')
plt.xlabel('Derived Importance Score')
plt.ylabel('Plasma Metabolites')

plt.xlim([0, data['importance'].max() * 1.1])
plt.xticks(rotation=45)

plt.savefig('met_severe_class_importance_plot_top15.png', dpi=300, bbox_inches='tight')

plt.show()